# QConfig

In [ ]:
import torch
from torchvision import models
import tvm
from tvm import relax
from tvm.relax.frontend.torch import from_exported_program
from torch.export import export

pipeline = tvm.transform.Sequential([
    relax.transform.FoldBatchnormToConv2D(),
    relax.transform.FoldConstant(),
    relax.transform.RemoveRedundantReshape(),
    # 将 BatchNorm 转换为一组更简单的算子以进行融合
    relax.transform.DecomposeOpsForInference(),
    # 规范化绑定
    relax.transform.CanonicalizeBindings(),

])

# 创建 PyTorch 模型
torch_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).eval()
# Give an example argument to torch.export
example_args = (torch.randn(1, 3, 224, 224, dtype=torch.float32),)
# Convert the model to IRModule
with torch.no_grad():
    exported_program = export(torch_model, example_args)
    mod = from_exported_program(exported_program, keep_params_as_input=False,)
run_mod = pipeline(mod)
run_mod = relax.get_pipeline("zero")(run_mod)

In [ ]:
run_mod.show()

In [7]:
from tvm.relax.frontend import nn

In [8]:
nn.extern

<function tvm.relax.frontend.nn.op.extern(name: str, args: Sequence[Union[tvm.relax.frontend.nn.core.Tensor, tvm.ir.expr.PrimExpr, int, float, str]], out: ~OutType) -> ~OutType>

In [12]:
from tvm import te
A = te.placeholder((128, 128), name="A")
B = te.placeholder((128, 128), name="B")
P = te.placeholder((1,), name="P")
C = te.extern(
    (128, 128),
    [A, B, P],
    lambda ins, outs: tvm.tir.call_extern(
        "", "myfunc", ins[0].data, ins[1].data, outs[0].data, ins[2][0]
    ),
    name="C",
)
func = te.create_prim_func([A, B, P, C])

In [13]:
func.show()